In [1]:
import pandas as pd
import numpy as np
import math
import time

from sklearn.metrics import classification_report, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import FunctionTransformer
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier

from nltk.lm.preprocessing import padded_everygram_pipeline, pad_both_ends
from nltk.util import trigrams
from nltk.lm import MLE

import warnings
warnings.filterwarnings('ignore')

In [2]:
df_train = pd.read_csv('../data/set-3/train/profile_info.csv').iloc[:, 1:]
df_label = pd.read_csv('../data/set-3/train/label.csv').iloc[:, 1:]
df_train = df_train.merge(df_label, on='ID')
df_train.head()

,ID,name,screen_name,location,description,url,protected,followers_count,friends_count,listed_count,...,profile_image_url_https,profile_link_color,profile_sidebar_border_color,profile_sidebar_fill_color,profile_text_color,profile_use_background_image,has_extended_profile,default_profile,default_profile_image,label
0,17461978,SHAQ,SHAQ,"Orlando, FL","VERY QUOTATIOUS, I PERFORM RANDOM ACTS OF SHAQ...",http://www.ShaqFuRadio.com,False,15349596,692,45568,...,https://pbs.twimg.com/profile_images/167390727...,2FC2EF,181A1E,252429,666666,True,False,False,False,0
1,1297437077403885568,Jennifer Fishpaw,JenniferFishpaw,,,NaN,False,0,44,0,...,https://pbs.twimg.com/profile_images/129743740...,1DA1F2,C0DEED,DDEEF6,333333,True,True,True,False,1
2,17685258,Brad Parscale,parscale,Florida,Owner @ Parscale Strategy. Senior Advisor Digi...,http://www.parscale.com,False,762839,475,3201,...,https://pbs.twimg.com/profile_images/129545322...,AB2316,FFFFFF,FFFFFF,666666,False,False,False,False,0
3,15750898,FOX 13 Tampa Bay,FOX13News,"Tampa, FL",Bringing you the important stuff like breaking...,http://www.FOX13news.com,False,327587,4801,1744,...,https://pbs.twimg.com/profile_images/129319301...,0B2F8A,FFFFFF,E8EEF0,333333,True,False,False,False,0
4,1659167666,Vonte The Plug 🎤🔌,VonteThePlugNC,"Jacksonville Beach, FL",MOTIVATION 3 OUT NOW 🔥 Singles: ‘Lil Shawdy’ &...,https://music.apple.com/us/artist/vonte-the-pl...,False,13324,647,44,...,https://pbs.twimg.com/profile_images/118166240...,1DA1F2,C0DEED,DDEEF6,333333,True,False,True,False,1


In [3]:
def get_screen_name_likelihood(series):
    sequence = series.apply(lambda x: list(x.lower())).values.tolist()
    return get_likelihood_array(sequence)
    
def get_likelihood_array(sequence, n=3):
    train_data, padded_sent = padded_everygram_pipeline(n, sequence)
    mle = MLE(n)
    mle.fit(train_data, padded_sent)
    
    s = np.zeros((len(sequence),))
    for i, name in enumerate(sequence):
        tri = trigrams(pad_both_ends(name, n=3))
        total_score = 1
        count = 0
        for ele in tri:
            score = mle.score(ele[2], [ele[0], ele[1]])
            total_score *= score
            count += 1
        s[i] = total_score ** (1/count)
    return s

def feature_engineering(df):
    used_columns = [
        'statuses_count',
        'followers_count',
        'friends_count',
        'favourites_count',
        'listed_count',
        'default_profile',
        'profile_use_background_image',
        'verified',
        'label'
    ]
    df_return = df[used_columns]
    age = (pd.to_datetime('2020-12-31 00:00:00') - pd.to_datetime(df['created_at']).dt.tz_localize(None)) / np.timedelta64(1, 'Y')
    df_return['tweet_freq'] = df['statuses_count'] / age
    df_return['followers_growth_rate'] = df['followers_count'] / age
    df_return['friends_growth_rate'] = df['friends_count'] / age
    df_return['favourites_growth_rate'] = df['favourites_count'] / age
    df_return['listed_growth_rate'] = df['listed_count'] / age
    df_return['followers_friends_ratio'] = df['followers_count'] / np.maximum(df['friends_count'], 1)
    df_return['screen_name_length'] = df['screen_name'].str.len()
    df_return['num_digits_in_screen_name'] = df['screen_name'].str.count('\d')
    df_return['name_length'] = df['name'].str.len()
    df_return['num_digits_in_name'] = df['name'].str.count('\d')
    df_return['description_length'] = df['description'].str.len()
    df_return['screen_name_likelihood'] = get_screen_name_likelihood(df['screen_name'])
    
    def bool_to_int(text):
        if 'True' in text:
            return 1
        elif 'False' in text:
            return 0
        else:
            return text
    for i in df_return.select_dtypes('object'):
        df_return[i] = df_return[i].apply(bool_to_int)
    
    return df_return.fillna(0.0)

In [4]:
feature_time = time.time()
df_new = feature_engineering(df_train)
end_feature_time = time.time()

In [5]:
df_new

,statuses_count,followers_count,friends_count,favourites_count,listed_count,default_profile,profile_use_background_image,verified,label,tweet_freq,...,friends_growth_rate,favourites_growth_rate,listed_growth_rate,followers_friends_ratio,screen_name_length,num_digits_in_screen_name,name_length,num_digits_in_name,description_length,screen_name_likelihood
0,9798,15349596,692,142,45568,0,1,1,0,808.630443,...,57.110866,11.719282,3760.734029,22181.497110,5,0,5,0,51,0.159735
1,0,0,44,7,0,1,1,0,1,0.000000,...,123.921859,19.714841,0.000000,0.000000,16,0,17,0,1,0.103708
2,5518,762839,475,953,3201,0,0,1,0,456.365283,...,39.284797,78.817708,264.738179,1605.976842,9,0,14,0,161,0.141913
3,192876,327587,4801,2946,1744,0,1,1,0,15553.281378,...,387.146684,237.561785,140.633997,68.233076,10,2,17,2,161,0.145170
4,103,13324,647,729,44,1,1,0,1,13.934062,...,87.527553,98.620690,5.952415,20.593509,15,0,18,0,104,0.073807
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8273,25790,10997,821,20115,221,1,1,1,0,3474.254335,...,110.599566,2709.756726,29.771625,13.394641,14,0,12,0,161,0.096893
8274,425,44,637,298,0,1,1,0,1,89.160821,...,133.636337,62.517470,0.000000,0.069074,14,2,13,0,158,0.106675
8275,1439,342,849,2464,0,1,1,0,1,129.783248,...,76.571214,222.227883,0.000000,0.402827,15,0,16,0,134,0.120930
8276,674,72,367,2634,2,1,1,0,1,92.151543,...,50.177472,360.129323,0.273447,0.196185,9,3,8,0,66,0.110918


In [6]:
X = df_new.drop('label', axis=1).values
y = df_new['label'].values

In [7]:
start_time = time.time()
transformer = FunctionTransformer(np.log1p, validate=True)
X = transformer.transform(X)

model = KNeighborsClassifier(9)
model.fit(X, y)
end_time = time.time()

In [8]:
y_predict = model.predict(X)

In [9]:
print(classification_report(y_predict, y, digits=4))

              precision    recall  f1-score   support

           0     0.6787    0.8367    0.7495      2946
           1     0.8965    0.7811    0.8348      5332

    accuracy                         0.8009      8278
   macro avg     0.7876    0.8089    0.7922      8278
weighted avg     0.8190    0.8009    0.8045      8278



In [10]:
# ROC AUC score
roc_auc_score(y_predict, y)

0.8089302748293996

In [11]:
# Training time and feature engineering time
end_time - start_time, end_feature_time - feature_time

(0.003995180130004883, 5.108443260192871)

In [12]:
df_test = pd.read_csv('../data/set-3/test/profile_info.csv').iloc[:, 1:]
df_label_test = pd.read_csv('../data/set-3/test/label.csv').iloc[:, 1:]
df_test = df_test.merge(df_label_test, on='ID')
df_test.head()

,ID,name,screen_name,location,description,url,protected,followers_count,friends_count,listed_count,...,profile_image_url_https,profile_link_color,profile_sidebar_border_color,profile_sidebar_fill_color,profile_text_color,profile_use_background_image,has_extended_profile,default_profile,default_profile_image,label
0,1188812492010487808,Sharon Israel ⭐️⭐️⭐️,SharonIsrael10,Los Angeles & Colorado,Day 1 Trump supporter. I rode the escalator! C...,NaN,False,16596,16944,1,...,https://pbs.twimg.com/profile_images/118883642...,1DA1F2,C0DEED,DDEEF6,333333,True,True,True,False,1
1,155659213,Cristiano Ronaldo,Cristiano,"Turim, Piemonte",This Privacy Policy addresses the collection a...,http://www.facebook.com/cristiano,False,87313765,50,83703,...,https://pbs.twimg.com/profile_images/115731332...,1643C9,FFFFFF,838387,0D0D0D,True,False,False,False,0
2,147725246,FoxNewsInsider,FoxNewsInsider,NYC,Stay connected with everything Fox - the lates...,http://insider.foxnews.com,False,161827,361,1471,...,https://pbs.twimg.com/profile_images/881932020...,0084B4,FFFFFF,DDEEF6,333333,True,False,False,False,0
3,1296248637194895360,El Realista,ElReali03271594,Puerto Rico,Aprendizaje. Pensamiento Crítico. Debate de id...,NaN,False,9,543,0,...,https://pbs.twimg.com/profile_images/129624930...,1DA1F2,C0DEED,DDEEF6,333333,True,True,True,False,1
4,1339835893,Hillary Clinton,HillaryClinton,"New York, NY","2016 Democratic Nominee, SecState, Senator, ha...",http://onwardtogether.org,False,28513011,846,40146,...,https://pbs.twimg.com/profile_images/129119233...,0057B8,000000,000000,000000,False,True,False,False,0


In [13]:
df_test = feature_engineering(df_test)
df_test

,statuses_count,followers_count,friends_count,favourites_count,listed_count,default_profile,profile_use_background_image,verified,label,tweet_freq,...,friends_growth_rate,favourites_growth_rate,listed_growth_rate,followers_friends_ratio,screen_name_length,num_digits_in_screen_name,name_length,num_digits_in_name,description_length,screen_name_likelihood
0,49757,16596,16944,42684,1,1,1,0,1,42319.675512,...,14411.330705,36303.897533,0.850527,9.794618e-01,15,2,21,0,158,0.131395
1,3569,87313765,50,2,83703,0,1,1,0,338.391012,...,4.740698,0.189628,7936.212627,1.746275e+06,10,0,18,0,103,0.125974
2,73786,161827,361,762,1471,0,1,1,0,6958.238288,...,34.043369,71.858856,138.719656,4.482742e+02,15,0,15,0,149,0.228812
3,2,9,543,0,0,1,1,0,1,5.493902,...,1491.594435,0.000000,0.000000,1.657459e-02,16,8,12,0,146,0.162052
4,11945,28513011,846,1123,40146,0,0,1,0,1545.868220,...,109.485518,145.333613,5195.514905,3.370332e+04,15,0,16,0,155,0.144202
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1178,2330,7760,8104,75,106,0,1,0,1,259.002717,...,900.840353,8.336997,11.782956,9.575518e-01,12,0,13,0,153,0.145644
1179,50168,8446,408,125825,55,1,1,0,1,13441.792359,...,109.317718,33712.994810,14.736457,2.070098e+01,15,0,48,4,144,0.131271
1180,2950,309,1961,1572,3,0,0,0,1,414.812710,...,275.744991,221.045959,0.421843,1.575727e-01,10,2,2,0,1,0.212090
1181,152,154,1019,627,0,1,1,0,1,18.681723,...,125.241288,77.062107,0.000000,1.511286e-01,10,2,10,0,1,0.146532


In [14]:
X = df_test.drop('label', axis=1).values
y = df_test['label'].values

In [15]:
X = transformer.transform(X)
y_predict = model.predict(X)

In [16]:
print(classification_report(y_predict, y, digits=4))

              precision    recall  f1-score   support

           0     0.6538    0.7819    0.7121       454
           1     0.8453    0.7421    0.7904       729

    accuracy                         0.7574      1183
   macro avg     0.7495    0.7620    0.7512      1183
weighted avg     0.7718    0.7574    0.7603      1183

